# Subset HydroSAR Stack

### Alex Lewandowski; Alaska Satellite Facility

This notebook subsets a stack of VV RTC and DEM GeoTiffs to an area of interest.

Subsetting may be performed with:
1. An interactive map with a drag/drop bounding box selector
1. Well-Known Text (WKT)
1. A shapefile

---

<div class="alert alert-info" style="display: flex; align-items: center; font-family: 'Times New Roman', Times, serif; background-color: #d1ecf1;">
  <div style="display: flex; align-items: center; width: 5%;">
    <a href="https://github.com/HydroSAR/HydroSAR/issues">
      <img src="https://opensarlab-docs.asf.alaska.edu/opensarlab-notebook-assets/logos/github_issues.png" alt="GitHub logo over the word Issues" style="width: 100px;">
    </a>
  </div>
  <div style="width: 95%;">
    <b>Did you find a bug? Do you have a feature request? Do you have questions about HydroSAR?</b>
    <br/>
    Explore GitHub Issues on this Jupyter Book's GitHub repository. Find solutions, add to the discussion, or start a new bug report or feature request: <a href="https://github.com/HydroSAR/HydroSAR/issues">HydroSAR Issues</a>
  </div>
</div>

<div class="alert alert-info" style="display: flex; align-items: center; justify-content: space-between; font-family: 'Times New Roman', Times, serif; background-color: #d1ecf1;">
  <div style="display: flex; align-items: center; margin-right: 10px; width: 5%;">
    <a href="mailto:uso@asf.alaska.edu">
      <img src="https://opensarlab-docs.asf.alaska.edu/opensarlab-notebook-assets/logos/ASF_support_logo.png" alt="ASF logo" style="width: 100px">
    </a>
  </div>
  <div style="width: 95%;">
    <b>Have a question related to SAR or ASF data access?</b>
    <br/>
    Contact ASF User Support: <a href="mailto:uso@asf.alaska.edu">uso@asf.alaska.edu</a>
  </div>
</div>

---
## 0. Select a directory holding an RTC stack prepared with the [Prepare a SAR RTC Data Stack for HydroSAR notebook](Prepare_HydroSAR_RTC_Stack.ipynb)

The directory should contain the following subdirectories:
- `VH`
- `VV`

In [ ]:
from pathlib import Path

from ipyfilechooser import FileChooser
from IPython.display import display

fc = FileChooser(Path.home())
display(fc)

In [ ]:
data_dir = Path(fc.selected_path)
vh_dir = data_dir / 'VH'
vv_dir = data_dir / 'VV'

vh_paths = sorted(list(vh_dir.glob('*VH.tif')))
vv_paths = sorted(list(vv_dir.glob('*VV.tif')))

---
## 1. Load the details of the data stack into a GeoDataFrame 

This assumes the data are in a single projection. 

In [ ]:
import geopandas as gpd
import sys

current = Path("..").resolve()
sys.path.append(str(current))
import util.util as util

gdf = gpd.GeoDataFrame(
    {
    'file': vh_paths + vv_paths,
    'data_type': ['VV_RTC' if 'VV' in str(pth) else 'VH_RTC' for pth in vh_paths + vv_paths],
    'SAR_acquisition_dt': util.get_dates(vh_paths + vv_paths),
    'EPSG': [util.get_epsg(p) for p in vh_paths + vv_paths],
    'geometry': [util.get_geotiff_bbox(p) for p in vh_paths + vv_paths],
    }
                  )
display(gdf)
if gdf['EPSG'].nunique() > 1:
    raise Exception(f'Error: Data is in {gdf["EPSG"].nunique()} projections.')

---
## 2. Select how to define your AOI for subsetting

In [ ]:
import opensarlab_lib as osl

subset_option = osl.select_parameter([
    'Draw a bounding box on a map',
    'Provide a polygon in Well-Known Text (WKT)',
    'Provide a shapefile'
])

display(subset_option)

---
## 3. Define a subset AOI

In [ ]:
%matplotlib widget

from datetime import datetime

from osgeo import ogr
from rasterio.warp import transform_bounds

draw = 'Draw' in subset_option.value
wkt_poly = 'WKT' in subset_option.value

max_extents = osl.get_max_extents(vv_paths)
xmin, ymin, xmax, ymax = transform_bounds(int(osl.get_projection(str(vv_paths[0]))), 3857, *max_extents)
max_extents = [xmin, ymin, xmax, ymax]

common_extents = osl.get_common_coverage_extents(vv_paths)
xmin, ymin, xmax, ymax = transform_bounds(int(osl.get_projection(str(vv_paths[0]))), 3857, *common_extents)
common_extents = [xmin, ymin, xmax, ymax]


if draw:   
    print('Maximum Extents: Pixels in this area are guaranteed to be included in at least one RTC in the stack.\n\n')
    
    print('Common Extents: Pixels in this area are guaranteed to included in every RTC in the stack.\n\n')
    
    print(f"Select an AOI inside the common area covered by the stack.")
    
    aoi = osl.AOI_Selector(max_extents, common_extents, figsize=(10, 8))
elif wkt_poly:   
    correct_wkt_input = False
    while not correct_wkt_input:
        wkt, wkt_shapely_geom = util.get_valid_wkt()
        wkt_ogr_geom = ogr.CreateGeometryFromWkt(wkt)
        
        if not util.check_within_bounds(wkt_shapely_geom, gdf):
            print('WKT exceeds bounds of at least one dataset')
            continue
    
        correct_wkt_input = True
        
    shp_path = data_dir / f'shape_{datetime.strftime(datetime.now(), "%Y%m%dT%H%M%S")}.shp'
    epsg = int(gdf.loc[gdf['data_type'] == 'VV_RTC'].iloc[0]['EPSG'])
    util.save_shapefile(wkt_ogr_geom, epsg, shp_path)
else:
    print('Select a shapefile (*.shp)')
    shp_fc = FileChooser(Path.home())
    display(shp_fc)

**If providing an AOI with WKT or a shapefile, confirm its location on a plot before subsetting**

In [ ]:
import contextily as ctx
from matplotlib.lines import Line2D
import matplotlib.pyplot as plt 
plt.rcParams.update({'font.size': 12})
from shapely.geometry import box

if not draw and not wkt_poly:
    shp_path = Path(shp_fc.selected)
    if shp_path.suffix != '.shp':
        raise Exception(f'Selected file suffix not ".shp"')
if not draw:   
    shp_gdf = gpd.read_file(shp_path)

    box1 = gpd.GeoDataFrame({"geometry": [box(*max_extents)]}, crs='EPSG:3857')
    box2 = gpd.GeoDataFrame({"geometry": [box(*common_extents)]}, crs='EPSG:3857')
    box1 = box1.to_crs(crs=shp_gdf.crs.to_string())
    box2 = box2.to_crs(crs=shp_gdf.crs.to_string())
    
    fig, ax = plt.subplots(1, 1, figsize=(10, 10))
    shp_gdf.plot(ax=ax, color='blue', alpha=0.5, edgecolor='k')
    box1.plot(ax=ax, color='none', edgecolor='red', linewidth=4)
    box2.plot(ax=ax, color='none', edgecolor='green', linewidth=4)
    
    ctx.add_basemap(ax, crs=shp_gdf.crs.to_string(), source=ctx.providers.OpenTopoMap)
    
    minx, miny, maxx, maxy = box1.total_bounds
    buffer_percentage = 0.05
    buffer_x = (maxx - minx) * buffer_percentage
    buffer_y = (maxy - miny) * buffer_percentage
    buffered_minx = minx - buffer_x
    buffered_maxx = maxx + buffer_x
    buffered_miny = miny - buffer_y
    buffered_maxy = maxy + buffer_y
    
    ax.set_xlim(buffered_minx, buffered_maxx)
    ax.set_ylim(buffered_miny, buffered_maxy)
    ax.set_title('Loaded Shape for Subsetting')
    
    legend_elements = [
        Line2D([0], [0], color='red', lw=4, label='Max area covered by data stack'),
        Line2D([0], [0], color='green', lw=4, label='Common area covered by data stack')
    ]
    ax.legend(handles=legend_elements)
    plt.show()
    print('Max area covered by data stack: Every pixel guaranteed to be present in at least 1 image')
    print('Common area covered by data stack: Every pixel guaranteed to be present in all images')

---
## 4. Subset the data

In [ ]:
from IPython.display import Markdown
from osgeo import gdal
from tqdm.auto import tqdm

for pth in tqdm(gdf['file']):
    print(f'Subsetting: {pth}')

    if draw:
        try:
            xmin, ymin, xmax, ymax = transform_bounds(
                3857,
                int(gdf['EPSG'].iloc[0]), 
                *[aoi.x1, aoi.y1, aoi.x2, aoi.y2]
            )
            ul = [xmin, ymax]
            lr = [xmax, ymin]
        except TypeError:
            print('TypeError')
            display(Markdown(f'<text style=color:red>This error may occur if an AOI was not selected.</text>'))
            display(Markdown(f'<text style=color:red>Note that the square tool icon in the AOI selector menu is <b>NOT</b> the selection tool. It is the zoom tool.</text>'))
        temp_pth = pth.parent/f'subset_{pth.name}'
        gdal.Translate(destName=temp_pth, srcDS=pth, projWin=[ul[0], ul[1], lr[0], lr[1]])
        pth.unlink()
        temp_pth.rename(pth)
    else:
        warp_args = gdal.WarpOptions(cutlineDSName=shp_path, cropToCutline=True) 
        gdal.Warp(pth, pth, options=warp_args)

*Subset_HydroSAR_Stack.ipynb - Version 1.0.0 - May 2024*